<h4><b>Importing necessary libraries and connecting to the database</b></h4>

In [185]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Connect to SQLite database
conn = sqlite3.connect("../saas.db")

<h4><b>Fetching Trial Account data from database</b></h4>

In [186]:
trial_acc = pd.read_sql("SELECT account_id,signup_date FROM accounts WHERE is_trial='True'",conn)
trial_acc["signup_date"]=pd.to_datetime(trial_acc["signup_date"])
trial_acc["window_start"]=trial_acc["signup_date"] #Made this column for name convinience
trial_acc["window_end"] = trial_acc["signup_date"] + pd.Timedelta(days=30)
trial_acc.head()

,account_id,signup_date,window_start,window_end
0,A-1f0ac7,2023-08-27,2023-08-27,2023-09-26
1,A-6c093d,2023-04-14,2023-04-14,2023-05-14
2,A-462d45,2024-01-15,2024-01-15,2024-02-14
3,A-832ec2,2023-11-19,2023-11-19,2023-12-19
4,A-00cac8,2023-09-15,2023-09-15,2023-10-15


<h4><b>Fetching subscriptions,Churn Date and Feature Usage data from the database</b></h4>

In [187]:
churn=pd.read_sql("SELECT account_id, MIN(churn_date) AS churn_date FROM churn GROUP BY account_id",conn)
churn["churn_date"]=pd.to_datetime(churn["churn_date"])
subs=pd.read_sql("SELECT subscription_id, account_id , start_date , end_date FROM subscriptions ",conn)
subs["start_date"]=pd.to_datetime(subs["start_date"])
subs["end_date"]=pd.to_datetime(subs["end_date"])
feature_usage=pd.read_sql("SELECT * FROM feature_usage",conn)
feature_usage["usage_date"]=pd.to_datetime(feature_usage["usage_date"])

<h4><b>Joining Trial Accounts data to Subscriptions data</b></h4>

In [188]:
trial_subs = trial_acc.merge(
    subs,
    on="account_id",
    how="left"
)
trial_subs = trial_subs[
    (trial_subs["start_date"] <= trial_subs["window_end"]) &
    (
        trial_subs["end_date"].isna() |
        (trial_subs["end_date"] >= trial_subs["window_start"])
    )
]
trial_subs.head()

,account_id,signup_date,window_start,window_end,subscription_id,start_date,end_date
15,A-6c093d,2023-04-14,2023-04-14,2023-05-14,S-0ff481,2023-04-30,NaT
21,A-462d45,2024-01-15,2024-01-15,2024-02-14,S-9dda85,2024-01-16,2024-05-29
22,A-462d45,2024-01-15,2024-01-15,2024-02-14,S-5a4fb5,2024-02-13,NaT
46,A-00cac8,2023-09-15,2023-09-15,2023-10-15,S-2e965d,2023-09-16,NaT
60,A-f446b6,2023-05-12,2023-05-12,2023-06-11,S-19b415,2023-05-28,NaT


<h4><b>Joining Trial Subscription data with Feature Usage data</b>(within the window period)</h4>

In [189]:
trial_usage = trial_subs.merge(
    feature_usage,
    on="subscription_id",
    how="left"
)

trial_usage = trial_usage[
    (trial_usage["usage_date"] >= trial_usage["window_start"]) &
    (trial_usage["usage_date"] <= trial_usage["window_end"])
]
trial_usage.head()

,account_id,signup_date,window_start,window_end,subscription_id,start_date,end_date,usage_id,usage_date,feature_name,usage_count,usage_duration_secs,error_count,is_beta_feature
0,A-6c093d,2023-04-14,2023-04-14,2023-05-14,S-0ff481,2023-04-30,NaT,U-42b2e6,2023-04-30,feature_16,10.0,1090.0,0.0,False
3,A-462d45,2024-01-15,2024-01-15,2024-02-14,S-9dda85,2024-01-16,2024-05-29,U-432eac,2024-01-16,feature_36,8.0,2336.0,0.0,False
5,A-462d45,2024-01-15,2024-01-15,2024-02-14,S-9dda85,2024-01-16,2024-05-29,U-b17cd4,2024-01-27,feature_14,10.0,1020.0,2.0,True
7,A-462d45,2024-01-15,2024-01-15,2024-02-14,S-5a4fb5,2024-02-13,NaT,U-a4e7f8,2024-02-13,feature_24,12.0,336.0,0.0,False
15,A-00cac8,2023-09-15,2023-09-15,2023-10-15,S-2e965d,2023-09-16,NaT,U-256c2c,2023-09-16,feature_23,15.0,825.0,1.0,False


<h4><b>Aggregating the Trial Account's Usage data to get Behavioral Features data in the 30 days window </b></h4>

In [190]:
behavioral_features = (
    trial_usage
    .groupby(["account_id","signup_date"])
    .agg(
        usage_events_30d=("usage_count", "sum"),
        active_days_30d=("usage_date", "nunique"),
        features_used_30d=("feature_name", "nunique"),
        total_errors_30d=("error_count", "sum"),
        total_usage_time_30d=("usage_duration_secs", "sum")
    )
    .reset_index()
)
#adding the churn date data to it
behavioral_features=behavioral_features.merge(
    churn,
    on="account_id",
    how="left"
)
behavioral_features.head()

,account_id,signup_date,usage_events_30d,active_days_30d,features_used_30d,total_errors_30d,total_usage_time_30d,churn_date
0,A-00bed1,2023-11-14,12.0,1,1,0.0,444.0,2024-01-03
1,A-00cac8,2023-09-15,25.0,2,2,1.0,6365.0,NaT
2,A-016043,2024-07-31,10.0,1,1,0.0,3070.0,2024-08-11
3,A-0f6450,2024-12-27,120.0,6,11,6.0,41729.0,2024-12-29
4,A-10b8f2,2023-01-11,8.0,1,1,1.0,3088.0,2024-06-04


<h4><b>Adding early churn feature to the behavioural data </b>(Churn within 90 days of signup is early churn in this case)</h4><p>NOTE: We used 90 days as measure because the steep retention drop was noticed in 1-3 months period, especially for trial accounts</p>

In [191]:
behavioral_features["early_churn"] = (behavioral_features["churn_date"]<=(behavioral_features["signup_date"]+pd.Timedelta(days=90))).fillna(0).astype(bool)
behavioral_features

,account_id,signup_date,usage_events_30d,active_days_30d,features_used_30d,total_errors_30d,total_usage_time_30d,churn_date,early_churn
0,A-00bed1,2023-11-14,12.0,1,1,0.0,444.0,2024-01-03,True
1,A-00cac8,2023-09-15,25.0,2,2,1.0,6365.0,NaT,False
2,A-016043,2024-07-31,10.0,1,1,0.0,3070.0,2024-08-11,True
3,A-0f6450,2024-12-27,120.0,6,11,6.0,41729.0,2024-12-29,True
4,A-10b8f2,2023-01-11,8.0,1,1,1.0,3088.0,2024-06-04,False
...,...,...,...,...,...,...,...,...,...
62,A-f446b6,2023-05-12,8.0,1,1,1.0,3344.0,NaT,False
63,A-f9cc74,2024-10-12,51.0,6,6,3.0,13479.0,NaT,False
64,A-fa2041,2024-05-29,16.0,2,2,1.0,1288.0,2024-07-13,True
65,A-fce879,2024-09-24,43.0,4,5,10.0,19993.0,2024-10-07,True


<h4><b>Calculating Early Churn Rate for low, medium and high activated users</b></h4>

<h5>Observing active-days quantiles to understand the distribution of active_days_30d</h5>

In [192]:
behavioral_features["active_days_30d"].quantile([0.25, 0.5, 0.75])

0.25    1.0
0.50    2.0
0.75    4.0
Name: active_days_30d, dtype: float64

<h5>Activation-level labeling using quantile-based thresholds</h5>

In [193]:
q1, q3 = behavioral_features["active_days_30d"].quantile([0.25, 0.75])

behavioral_features["activation_level"] = pd.cut(
    behavioral_features["active_days_30d"],
    bins=[0, q1, q3, behavioral_features["active_days_30d"].max()],
    labels=["low", "medium", "high"]
)

<h5>Claculating early churn based on activation level</h5>

In [194]:
behavioral_features.groupby("activation_level")["early_churn"].mean()
# behavioral_features.groupby("activation_level")["account_id"].count()

activation_level
low       0.178571
medium    0.400000
high      0.642857
Name: early_churn, dtype: float64

<h5>More active users have higher likelihood of having early churn </h5>

<h4><b>Calculating Early Churn Rate for High Usage and Low Usage Accounts</b></h4>

In [195]:
behavioral_features["high_usage"]=(behavioral_features["total_usage_time_30d"]>=behavioral_features["total_usage_time_30d"].median()).astype(bool)
high_usage_churn = behavioral_features.groupby("high_usage")["early_churn"].mean().reset_index(name="early_churn_rate")
high_usage_churn

,high_usage,early_churn_rate
0,False,0.181818
1,True,0.529412


<h5>High Usage users have higher likelihood of churning early</h5>

<h4><b>Investigating the reason for power user's high early churn rates</b></h4>

<h5>Calculating mean, median error in the 30 days per activation group and the count of each activation group</h5>

In [196]:
behavioral_features.groupby("activation_level").agg(
        avg_errors_30d=("total_errors_30d", "mean"),
        median_error_30d=("total_errors_30d","median"),
        users=("account_id", "count")

    ).reset_index()

,activation_level,avg_errors_30d,median_error_30d,users
0,low,0.464286,0.0,28
1,medium,1.960000,1.0,25
2,high,4.500000,3.5,14


<h4><b>Calculating errors per active day for each activation group</b></h4>

In [197]:
behavioral_features["errors_per_active_day"] = (
    behavioral_features["total_errors_30d"] /
    behavioral_features["active_days_30d"].replace(0, 1)
)

behavioral_features.groupby("activation_level")["errors_per_active_day"].mean().reset_index(name="avg_errors_per_active_day")

,activation_level,avg_errors_per_active_day
0,low,0.464286
1,medium,0.723333
2,high,0.664201


<h5>
The higher early churn observed among power users appears to be driven by increased error encounters. 
Greater usage and higher activation levels increase the likelihood of encountering issues. 
Given this pattern, we further investigate error resolution to better understand its impact on churn.
</h5>

### Conclusion
- Behavioral cohort analysis reveals a friction-driven churn dynamic rather than a traditional activation failure. Early churn is primarily associated with high engagement combined with elevated error exposure, rather than insufficient usage.

- These findings suggest that retention efforts should prioritize reducing early product friction for engaged trial users, rather than simply increasing activation or usage volume.

<h4>Saving the behavioural_features file as csv for modelling</h4>

In [198]:
behavioral_features.to_csv("../data/processed/trial_behavioral_features.csv",index=False)